# Machine Learning Models
#### A gentle but rushed introduction

Welcome to this interactive notebook where give a brief introduction, followed by a practical examples, about how machine learning models work. We'll cover all of the nitty-gritty details starting from data loading and cleaning, choosing and creating a model, fitting it to data and evaluating it. The idea is for you to read along and run each cell one at the time, sometimes you'll have to change and fill some of the missing code for the cell to run properly. As alway the internet is your friend, so google all everything you find confusing. We've left some link we find usefull.

And naturally, feel free to ask for any help and advice! :)

### Imports
In this next cell we `import` all of the packages that contain functionalities we'll find usefull in this course.

In [18]:
import json
import numpy as np
import sklearn

import numpy.random as random
import matplotlib.pyplot as plt
from numpy.polynomial.polynomial import Polynomial

## 1.) High level overview of Machine Learning
In machine learning we typically attempt to solve problems or automate processes that require a lot of tiresome human work. Some of these tasks might be easy for humans but the task simply demands a lot of work and needs to be automated (e.g. retyping handwritten text into a computer), or something humans are also poor at but the task is important (e.g. determining weather a patient is likely to develop some disease). Regardless of the task difficulty we assume that we have some (preferably a lot) of data about the problem in hand. A collection of this data we call a dataset and here we will refer to it as $\mathcal{D}$. This dataset usually consists from examples of solutions to the problem we are trying to solve. For instance if we are trying to predict house prices given a dataset of already sold houses, the dataset $\mathcal{D}$ will consist of pairs like $(x,y)$, where $y$ is the price that the house was sold for. Usually we call variables such as $y$ (variables we're trying to guess): *targets*. A variable $x$ is the so-called *feature vector*, it consists of all the aditional information we may have about the house before it was sold. $x=($`house_area`,`number_of_rooms`,`location`,`age`...$)$. The goal of machine learning is to create some procedure such that given any $x$ (that may or may not be present in $\mathcal{D}$) we can predict the correct $y$.

### 1.1) Machine learning pipeline
The above mentioned procedure usually goes in the following steps
1. Data acquisition.
2. Data preparation (data cleaning, and feature extraction).
3. Model selection.
4. Model fitting (training).
5. Model evaluation (testing).

Here we'll assume that step $1.$ has been done for us ahead of time, we'll briefly touch on step $2.$ and focus mostly on steps $3.-5.$

### 1.2) Types of machine learning tasks
In most cases we'll encounter machine learning problem can be divided into two broad categories:
- Regression tasks
- Clasification tasks

In both cases we have a dataset such as $\mathcal{D} = \{ (x_i,y_i) \}| i=1,2...N \}$ (Here we only note that we have $N$ different examples in the dataset, the subscript $i$ notes an arbitrary example), the only difference comes in the type of the target variable $y_i$. In regression tasks $y_i$ is a real number value, and the task is to approximate the correct value of $y_i$ as close as possible, an example of this is the house price prediction: the target $y_i$ is expressed in euros, dollars, or some other currency. The problem of detecting diseases among patients is a classification task. Here's why: each patient has some features associated with them $x_i =($`age`,`sex`,`blood_sugar_levels`,...$)$ and needs to be classified in one of the two categories: $y_i=$`has_disease` or $y_i=$`healthy`. If $y_i$ can have only two values we call these types of problems: binary calssification, naturally we can encounter problems where $y_i$ can have many discrete values (we call that simply a classification problem). 

### 1.3) Regression problems and linear regression
As our starting example we'll generate some fake, and overly simple data just so we can introduce the basic concepts of regression models (most of these will be present in classificatio tasks with only sight modifications). In the cell below we created some random data, that might correspond to house prices, given the house total area (note that these houses are pretty small and that the price is in Monopoly money so this is far from a realistic example). But suppose that in this dream scenario we find that the house price can be modeled precisely by only using the information about its area. In the graph below the blue dots correspond to measured (observed) house prices and areas, our goal is to create a model that best approximates this data so that in some future we might find a house with some `area`, plug that area into our model and know it's market price before it is even sold, neat! But how do we do this?

The modeling happens in several steps: we first decide what kind of function we will be using to model our data, this function is usually defined by some parameters that can be tuned to fit the data (in this case our function will be a line, with tunable parameters: slope $A$ and intercept $B$). Okay, in the cell below we have defined one such line, and its predictions can be seen on the orange line below, and it seems our model is failing disasterously...

Here comes the next step **model fitting**: The idea is to tell the model how *wrong* it is on the data. We do this by defining a loss (or sometimes error) function $\mathcal{L}$ it depends only on the model parameters, and the best model will be the one that has the minimal loss. But what is this loss function? It's shape can vary greatly depending on the problem we're facing but when it comes to regression, a good choice is *Mean Square Error* (MSE).

$$
\mathcal{L}_{MSE} = \frac{1}{N}\sum_i^N (f_m(x_i) - y_i)^2
$$
In the formula above the symbol $f_m$ represents our model and $f_m(x_i)$ is the model evaluated at some $i$-th datapoint $x_i$, while $y_i$ is the target of the feature $x_i$. In other words we want to minimize the square distance of our model prediction to the real data.

The actual minimization is done by sophisticated alogrithms that some clever people have developed before hand, what is important to take from here is that given a nice loss function there are algorithms that find the best model parameters. We'll discuss this some more in the next section. 

If you are stil curious about how we minimize this loss function we encourage to check out gradient descent:

https://www.geeksforgeeks.org/gradient-descent-algorithm-and-its-variants/

https://arxiv.org/pdf/1609.04747


#### Excercise 1. Try fixing the model!
In the cell below you'll find that our model is very poor, try to fix it! Try playing around with the values of `A_param`, and `B_param`, and see how the loss changes. The closer you are to the optimal parameters the loss will be smaller!

In [ ]:
def get_line_measurements(num_samples, noise_strength=2):
    x = random.uniform(low=1, high=7, size=num_samples)
    return x, 2.2 * x + 1 + noise_strength * random.normal(size=num_samples)


def get_line_fit(x, A, B):
    """
    This is a very simple regression model.
    It is in fact a straight line but a regression
    model in its own right. Most of what we discuss
    for this model will apply to more complicated models.
    """
    return A * x + B


def plot_fake_regression(A, B, num_samples=100, noise_strength=1, model=lambda x: x):
    xx = np.linspace(1, 7, 100)

    num_samples = 100
    x, y = get_line_measurements(num_samples, noise_strength=noise_strength)

    y_fit = model(xx)
    error = ((model(x) - y) ** 2).mean()
    plt.scatter(x, y, alpha=0.8, s=10, label="data points")
    plt.plot(xx, y_fit, linewidth=2, color="tab:orange", label="model prediction")
    plt.title("Average square error: {:.2f}".format(error))
    plt.ylabel("House price in Monopoly Money")
    plt.xlabel("House area in square meters.")
    plt.legend()


# Change the values of A, and B
# to get a lower error
A_param = -3
B_param = 22
dummy_model = lambda x: get_line_fit(x, A_param, B_param)
plot_fake_regression(A_param, B_param, model=dummy_model)

## 2.) Regression and `scikit learn`
In the previous section you've learned how to crawl, now let's try to walk and, hopefuly by the end of this tutorial, run. Due to the explosive growth of the machine learning industry, we have now a plethora of well documented, optimized and reliable libraries to do the boring part of training our models. One such library is `scikit learn` or `sklearn` it's documentation is truly a gem and we warmly recommend that you check it out once you feel ready to leave the bay and head to the machine learning depths: https://scikit-learn.org/1.5/index.html

### 2.1) Fitting the model automatically
In the cell before you were required to do the job algorithms usually do, that is tweaking the model parameters so the model can better fit the data. Here we'll first show `sklearn` can to that for us. We again generate random data $x,y$, next we initialize a `linear_model` using `sklearn` it essentialy does the same calculations as we did above, except it's a bit more self aware and knows how to optimize its parameters given some data. we do this by calling the `fit` function. Once the `fit` function has done its job we use the `predict` method to generate model predictions

In [ ]:
# Again we first generate some fake house data
import sklearn.linear_model


x, y = get_line_measurements(num_samples=100)


linear_model = sklearn.linear_model.LinearRegression()

linear_model.fit(x[:, None], y)


xx = np.linspace(x.min(), x.max(), 100)
yy = linear_model.predict(xx[:, None])
plt.scatter(x, y, label="data points", s=7)
plt.plot(xx, yy, label="model precitions", color="tab:orange", linewidth=2)
plt.legend()

### 2.2) Digression: Many features
In all examples so far our data consisted of pairs of $(x,y)$ where both of these were just numbers. This implies that the all the necessary information to model the behaviour of the target $y$ is kept in $x$. If only the real world were so simple...

In practice each $x$ is a vector, that is it consists of multiple components, each representing some information we have about the data in question (remeber the house example). In the cell below we'll show how `sklearn` can handle multidimensional data just as easily as it handled single dimensional data in examples above.

In the next cell we'll denote our feature vector with the capital $X$ to indicate it is multidimensional, in python we represent these with arrays. A typical frame work for dealing with arrays is called `numpy` which in this notebook is aliased as `np`. Numpy is also one of those wonders why people from all over abandon other programming languages and cling to Python. It is well maintained, well documented, and easy to use. We ecnourage to look at its amazing documentation: https://numpy.org/doc/stable/index.html

In the cell below we will simply perform the fitting of the data... We initalize the array $X$ that has dimensions $($`num_samples`$\times$`num_dims`$)$, the target $y$ is again a real number. We define the model just as before and fit it to the data.

Mathematically the only difference is that the model no longer calculates: 
$$y = A x + B$$
Rather, it has more parameters (`num_dims`$+1$ to be exact) that all need to be fitted to the data
$$y = A_1 X_1 + A_2 X_2 + ...+ A_n X_n + B$$


In [ ]:
num_samples = 100
num_dims = 5
X = np.random.randn(num_samples, num_dims)
y = np.random.randn(num_samples, 1)
print(f"To see the structure of the array X we use the .shape attribute {X.shape}")
print(f"You can see X has the shape num_samples x num_dims: {num_samples}x{num_dims}")
multidim_linear_model = sklearn.linear_model.LinearRegression()

multidim_linear_model.fit(X, y)

print(
    f"Once the model is trained you can use this attribute to find its parameters: {multidim_linear_model.coef_}"
)

### 2.3) Data that can't be fit by a line
Okay so now we understand how to fit a model to any data so long as it can be fit by a line (or a hyperplane for higher dimensions), but what if the data cannot be fit using a line? In the code below we'll make use of the `numpy`s `Polynomial` object, we pass it $n+1$ coefficients and it gives us a polynomial of order $n$ with those coefficents. In the cell below we have arbitrarily chosen these coeficients to be (1,-2,3), and undereath the cell you can see its mathematical formula. You are encouraged to try, though you will fail, to fit a linear model to this data. It will always fail in some way. The behaviour shown below is called underfitting. 

Before going to the next cell think is there a way to make use of multiple dimensions to fit this data using linear regression?

In [ ]:
x = np.random.uniform(-5, 5, 100)
curve_generator = Polynomial((1, -2, 3))
y_true = curve_generator(x) + 2 * np.random.randn(x.shape[0])
model = sklearn.linear_model.LinearRegression()
model.fit(x[:, None], y_true[:, None])
xx = np.linspace(-5, 5, 200)
y_model = model.predict(xx[:, None])

plt.plot(xx, y_model, color="tab:orange", label="model prediction")
plt.scatter(x, y_true, s=10, label="data")
plt.title("Nonlinear data")
plt.legend()
curve_generator

### 2.4) Solution: Fitting linear models to nonlinear data
Recall from section (2.2) how we defined a linear model that handles features with multiple dimensions.
$$y = A_1 X_1 + A_2 X_2 + ...+ A_n X_n + B$$
While modeling we are not forced to use $x$ and only $x$ we can (and most often do) transform it in some way to better capture some patterns in the data. Here our $x$ is 1-dimensional, but we will make it $n$-dimensional.

We'll try the following, instead of using our 1-dimensional to model the data, we will use an $n$-dimensional $X$ for modeling, but this capital $X$ is constructed from the regular $x$ in the following way:
$$X = (x,x^2,x^3,...,x^n)$$
Note $n$ is something we, as engineers have to choose. Now that we have $X$ we will plug it into our linear model and get nonlinear outputs, using the above equation we can get 
$$y = A_1 x + A_2 x^2 + ...+ A_n x^n + B$$
This is called *polynomial regression*. 

**NOTE:** Parameters that engineers have to choose (such as *polynomial order*) are called hyperparameters.

### Excercise 3)
Follow the commented lines of code and for step by step instructions, on how to complete the exercise

What happens when we choose a ridiculously high polynomial order? Say, $n=12$ ...


In [ ]:
x = np.random.uniform(-5, 5, 100)
curve_generator = Polynomial((1, -2, 3))
y_true = curve_generator(x) + 2 * np.random.randn(x.shape[0])

# MINI TASK 1
# Modify X below so you choose the correct n.
# Here n is set to 5.
X = [x, x**2, x**3, x**4, x**5]
# MINI TASK 2
# Define and fit a linear model on
# polynomial data X
# Your code here:
################


################
X = np.concatenate([x[:, None] for x in X], axis=1)
xx = np.linspace(-5, 5, 200)
XX = np.concatenate([xx[:, None] ** i for i in range(1, X.shape[1] + 1)], axis=1)

y_model = model.predict(XX)
plt.plot(xx, y_model, color="tab:orange", label="model prediction")
plt.scatter(x, y_true, s=10, label="data")
plt.title("Nonlinear data")
plt.legend()

#### Excercise 4) No more training wheels
Before we always generated the data right infront of you, so you could sort of cheat your way through the excercise and see which are the correct parameters. Now we will load some chaotic data that we've simulated and it is your job to fit several polynomial models to the data.
Vary the order of the polynomial, try plotting the model predictions, try describing the model performance...

Hopefuly, you'll see how picking a proper model design, and its hyperparemeters is sometimes more an art than science. The best decisions are usually based upon intuition, and experience.

In [ ]:
# Loading data
with open("data/lorenz.json", "r") as f:
    all_data = np.array(json.load(f))
t = np.arange(0, 10, 5e-3)
idx = np.random.randint(low=0, high=len(t), size=500)
y = all_data[5, idx, 1]
x = t[idx]

################
# Your code here
################


################

plt.scatter(x, y, s=10, label="data")
plt.title("Very nonlinear data")

## 3.) Classification problems
In this third section we'll use the knowledge we've built up in previous sections to solve a simple binary classification task. Lucky for us, `sklearn` has a large number of toy datasets, perfect for learning. We make a simple binary classifier on the Breast Cancer Dataset:

 https://scikit-learn.org/1.5/datasets/toy_dataset.html#breast-cancer-dataset

In the dataset we are $569$ examples of patients, each patient has $30$ features ascribed to them (a more deatiled desctiption on what those features are can be found on the link above). The target $y$ is an array containing ones and zeros, a zero means no breast cancer, while 1 means breast cancer. Our goal is to make a model that can predict weather a patient has breast cancer automatically.

### 3.1) Data splitting
Previously we haven't been fully honest, in the sense that we've fitted the model on the entire data. Why is this bad? Because we have no way of telling how will the model perform on unseen data. To do this we take our data set and split it into two parts a *training set* and *validation set*. The former is used for fitting the model parameters, while the latter is used only to test how the model works on new data.

Usually a rule of thumb is to divide the data into $0.7$ - $0.3$ ratio, so $70$% to the training set and $30$ to the validation set (or in 0.8-0.2 ratio).

In the cell below we load the data and perform this splitting.

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

dataset = load_breast_cancer()
X = dataset["data"]
y = dataset["target"]
print(f"Full dataset shape X: {X.shape} y:{y.shape}")
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
print(f"Train dataset shape X_train: {X_train.shape} y_train: {y_train.shape}")

print(f"Validation dataset shape X_val: {X_val.shape} y_val: {y_val.shape}")

### 3.2) Data cleaning and feature engineering
After the data has been split properly, we perform some data engineering, that is we look at what the data actually represents and try to combine it in a more meaningful way. We remove some outliers, normalize features, and figure out how to treat the data properly.
More can be found on:

https://scikit-learn.org/stable/modules/preprocessing.html

We will skip this part, and leave it for your curiosity. The reason is that this is usually the longest and most tedious part of machine learning, and it is typically problem specific, so there are a very few general points to take away from here. If you feel comfortable with doing this we encourage you to try playing around with your data. In the following days we can make an extended tutorial on the methods common in this step if you'd like!


In [ ]:
# Data cleaning and feature engineering
# Your code here ...

### 3.3) How to classify?
As you may have noticed our models output real valued continuous values, but here we need something that outputs $0$ or $1$, how do we do this? The trick is to take the model output $\hat{y}$ and act on it with a functin that squishes it value between $0$ and $1$. One such function is this
$$
\sigma(x) = \frac{1}{1+e^{-x}}
$$
Once we've done this we can treat the output of the model as a probability of the target being $1$, we also define some probability threshold usually $0.5$, above which we assume the model prediction to be $1$.

Also in classification the loss function changes a bit, in fancy math terms we say that: *We minimize the negative log likelyhood*, and the formula for the loss is
$$
\mathcal{L} = \frac{1}{N}\sum_{i=1}^{N} y_i \log p_m(x_i) +  (1-y_i)\log(1-p_m(x_i))
$$
We've changed the notation a bit, now $p_m(x_i)$ is the output of our model since it predicts probabilites. If all of this seems scary, do not worry `skelarn` and other libraries take care of this for you, we just wanted to show you how this works under the hood.ž
In the cell below we define the classifier and fit it to data, just as before.

Model documentation:

https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.LogisticRegression.html#


In [ ]:
classifier = sklearn.linear_model.LogisticRegression()
classifier.fit(X_train, y_train)

### 3.2) Model evaluation
Our model has been trained now it is time to test it on some data it has not yet *seen*. To do this we will use the model to make predictions on `X_val` and compare the model results with `y_val`.

### Excercise 4)
Try writing the code to get the model predictions yourself. Store the results in the `y_pred` matrix.

In [ ]:
# Your code here...
####################

### 3.3) Model metrics
To see how well or poorly the model is doing we use several metrics evaluate it. In classification there are several most comon ones: `accuracy_score`,`precision_score`, `recall_score`, and the `f1_score`. Each of them can be extracted from the so-called `confusion_matrix`.
The confusion matrix is essentially a table, the rows are what our model predicts, and the columns are what the actual targets are. Below we show an example confusion matrix:
1. First row, first column: the data is actually $1$ and the model prediction is $1$, so this is a true positive (TP).
2. First row, second column: the data is actually $0$ and the model prediction is $1$, so this is a false positive (FP).
3. Second row, first column: the data is actually $1$ and the model prediction is $0$, so this is a false negative (FN).
4. Second row, second column: the data is actually $0$ and the model prediction is $0$, so this is a true negative (TN).


|   | `y_true` $= 1$  |  `y_true` $= 0$  |  
|---|---|---|
| `y_pred` $=1$  | (TP) 700  |  (FP) 32 |   
|  `y_pred` $=0$ |  (FN) 32 | (TN) 200  |   

All of the metrics can be computed from the confusion matrix:
$$
Accuracy = \frac{TP + TN}{TP+TN+FP+FN}
$$

$$
Precision = \frac{TP}{TP+FP}
$$

$$
Recall = \frac{TP}{TP+FN}
$$

$$
F1 = \frac{2 \times TP }{2\times TP+TN+FP+FN}
$$

### Excercise 5)
You task in the code bellow is to use `y_valid` array and the array of predictions you got in the previous excercise to calculate the confusion matrix, accuracy, precision, recall and $F1$ score. Report on how the model is performing.

In [ ]:
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)

####################
# Your code here
####################


####################